In [2]:
%run Animal.ipynb
%run Food.ipynb

from abc import ABC, abstractmethod
from copy import deepcopy
from numpy.random import choice

In [3]:
class Container(ABC):
    @abstractmethod
    def __init__(self, pets = []):     
        pass
        
    @abstractmethod
    def __setitem__(self, key, pet):
        pass
        
    def __getitem__(self, key):
        return self.pets[key]
    
    def __iter__(self):
        return iter(self.pets)
    
    def __len__(self):
        return len(self.pets)
    
    def __repr__(self):
        if len(self.pets) == 0:
            return 'Empty Team'
        return '[' + '\n'.join([repr(item) for item in self]) + ']'

In [4]:
class Team(Container):
    """
    Team object designed to contain animals and two trigger dictionaries for battle and the shop.
    """
    max_size = 5
    
    def __init__(self, pets=[]):
        for pet in pets:
            assert isinstance(pet, Pet), f'{pet} is not a Pet and cannot be in a Team!'
        self.pets = pets
        self.dead_pets = []
        
        for i, pet in enumerate(pets):
            pet.pos = i
            
    @property
    def current_turn(self):
        return self._current_turn
    @current_turn.setter
    def current_turn(self, new_val):
        self._current_turn = new_val
        
            
    def __setitem__(self, key, pet):
        assert isinstance(pet, Pet), f'{pet} is not a Pet and cannot be in a Team!'
        assert key in range(len(self)), f'Tried to set animal into slot {key} but team was only length {len(self)}!'
        self.pets[key] = pet
    
    def insert(self, animal, pos = max_size-1):
        assert len(self) < Team.max_size, 'Team is already max size!'
        assert isinstance(animal, Pet), f'{animal} is not a Pet and cannot be in a Team!'
        self.pets.insert(pos, animal)
        for i in range(len(self)):
            self[i].pos = i
        for pet in self.dead_pets:
            if pos <= pet.pos:
                pet.pos += 1
    
    def insert_by_name(self, species, lvl=1, atk=None, hp=None, status=None, exp=0, pos=4):
        to_add = Pet(species, lvl, atk, hp, status)
        self.insert(to_add, pos)
        
#     def copy(self):
#         return Team([pet for pet in self])

    def deepcopy(self):
        return Team([deepcopy(pet) for pet in self])
    
    def battle_copy(self):
        copied = deepcopy(self)
        for pet in copied:
            pet.atk += pet.temp_atk
            pet.hp += pet.temp_hp
            pet.temp_atk = 0
            pet.temp_hp = 0
        return copied
            
    def drop(self, pos=-1):
        assert pos < len(self.pets), f'remove_pet tried to drop at {pos} but team length was only {len(self)}!'
        removed = self.pets.pop(pos)
        for i in range(pos, len(self)):
            self[i].pos = i
        for pet in self.dead_pets:
            if pos < pet.pos:
                pet.pos -= 1
        return removed
    
    def exclude(self, pos):
        return [self[i] for i in range(len(self)) if i != pos]
        
    def shuffle(self):
        new_inds = np.random.choice(range(len(self)), size=len(self), replace=False)
        new_pets = []
        for i, new_ind in enumerate(new_inds):
            pet = self.pets[new_ind]
            pet.pos = i
            new_pets.append(pet)
        self.pets = new_pets
        return self
    
    def switch_pets(self, index1, index2):
        assert index1 in range(self.team.size()), 'switch_pets got a bad index1: {}'.format(index1)
        assert index2 in range(self.team.size()), 'switch_pets got a bad index2: {}'.format(index2)
        assert index1 != index2, 'must switch pets at different indices'
        
        self[index1], self[index2] = self[index2], self[index1]
        self[index1].pos = index1
        self[index2].pos = index2
        
def random_teams():
    species_list = pets_df[pets_df.Rollable].index.get_level_values(0).unique()
    
    team1_size, team2_size = choice([3, 4, 5], 2)
    team1 = Team([Pet(choice(species_list), lvl=choice([1, 2, 3]), 
                         atk=choice(range(1, 21)), hp=choice(range(1, 21)), 
                         status=choice(status_list)) for i in range(team1_size)])
    team2 = Team([Pet(choice(species_list), lvl=choice([1, 2, 3]), 
                         atk=choice(range(1, 21)), hp=choice(range(1, 21)), 
                         status=choice(status_list)) for i in range(team2_size)])
    
    return team1, team2

In [5]:
class ShopTeam(Container):
    def __init__(self, animal_dict = pack1_animal_dict):
        self.animal_dict = animal_dict
        self._perm_atk_buff = 0
        self._perm_hp_buff = 0
        
        self.pets = []
        self.restock(tier=1, n_pets=3)
        
    @property
    def perm_atk_buff(self):
        return self._perm_atk_buff
    @perm_atk_buff.setter
    def perm_atk_buff(self, new_val):
        diff = new_val - self._perm_atk_buff
        assert diff > 0, "Can only increase ShopTeam's perm_atk"
        for pet in self:
            pet.atk += diff
        self._perm_atk_buff = new_val
        
    @property
    def perm_hp_buff(self):
        return self._perm_hp_buff
    @perm_hp_buff.setter
    def perm_hp_buff(self, new_val):
        diff = new_val - self._perm_hp_buff
        assert diff > 0, "Can only increase ShopTeam's perm_hp"
        for pet in self:
            pet.hp += diff
        self._perm_hp_buff = new_val
        
    def __setitem__(self, key, pet):
        assert isinstance(pet, ShopAnimal), f'{pet} is not a ShopAnimal and cannot be in a ShopTeam!'
        assert key in range(len(self)), f'Tried to set animal into slot {key} but team was only length {len(self.team)}!'
        self.pets[key] = pet
        
    def restock(self, tier, n_pets):
        frozen_pets = [pet for pet in self.pets if pet.is_frozen]
        if len(frozen_pets) < n_pets:
            possible = np.concatenate([self.animal_dict[i] for i in range(1, tier+1)])
            pet_list = [ShopAnimal(species) for species in choice(possible, size=n_pets-len(frozen_pets), replace=True)]
            for pet in pet_list:
                pet.atk += self.perm_atk_buff
                pet.hp += self.perm_hp_buff
            self.pets = frozen_pets + pet_list
        
    def drop(self, pos=-1):
        to_drop = self.pets.pop(pos)
        to_drop.atk += self.perm_atk_buff
        to_drop.hp += self.perm_hp_buff
        return to_drop
        
    def add_pet_by_name(self, name):
        to_add = ShopAnimal(name)
        to_add.atk += self.perm_atk_buff
        to_add.hp += self.perm_hp_buff
        
        self.pets.append(to_add)
        
    def add_pet_by_tier(self, tier):
        possible = np.concatenate([self.animal_dict[i] for i in range(1, tier+1)])
        self.add_pet_by_name(choice(possible))
    

In [6]:
class ShopFoods(Container):
    def __init__(self, food_dict = pack1_food_dict):
        self.food_dict = food_dict
        self.pets = []
        
        self.restock(tier=1, n_foods=1)
        
    def __setitem__(self, key, food):
        assert isinstance(food, Food), f'{food} is not a Food and cannot be in ShopFoods!'
        assert key in range(len(self)), f'Tried to set food into slot {key} but ShopFoods was only length {len(self.pets)}!'
        self.pets[key] = food
        
    def restock(self, tier, n_foods):
        assert tier in range(1, 7)
        frozen_foods = [food for food in self.pets if food.is_frozen]
        if len(frozen_foods) < n_foods:
            possible = np.concatenate([self.food_dict[i] for i in range(1, tier+1)])
            self.pets = frozen_foods + [Food(name) for name in choice(possible, size=n_foods-len(frozen_foods), 
                                                                      replace=True)]
        
    def drop(self, pos):
        return self.pets.pop(pos)
    
    def add_food_by_name(self, name):
        self.pets.append(Food(name))
        
    def add_food_by_tier(self, tier):
        assert tier in range(1, 7)
        possible = np.concatenate([self.food_dict[i] for i in range(1, tier+1)])
        self.add_food_by_name(choice(possible))
        
    def replace_all_foods(self, food_list):
        self.pets = [Food(name) for name in food_list]
        
        
    
        